In [7]:
#Required only if dataframe_image is not already installed.
!pip install dataframe_image

In [10]:
LOCAL=True

if LOCAL == False:
   stub = modal.Stub()
   hopsworks_image = modal.Image.debian_slim().pip_install(["hopsworks","joblib","seaborn","sklearn==1.1.1","dataframe-image"])
   @stub.function(image=hopsworks_image, schedule=modal.Period(days=1), secret=modal.Secret.from_name("HOPSWORKS_API_KEY"))
   def f():
       g()

def g():
    import pandas as pd
    import hopsworks
    import joblib
    import datetime
    from PIL import Image
    from datetime import datetime
    import dataframe_image as dfi
    from sklearn.metrics import confusion_matrix
    from matplotlib import pyplot
    import seaborn as sns
    import requests

    project = hopsworks.login()
    fs = project.get_feature_store()
    
    mr = project.get_model_registry()
    model = mr.get_model("sp500_model", version=2)
    model_dir = model.download()
    model = joblib.load(model_dir + "/sp500_model.pkl")
    
    feature_view = fs.get_feature_view(name="sp500", version=2)
    batch_data = feature_view.get_batch_data() 
    batch_data = batch_data.sort_values(by='date_copy')
    batch_data = batch_data.reset_index(drop=True)
    
    batch_data["tomorrow"] = batch_data["close"].shift(-1)
    batch_data["target"] = (batch_data["tomorrow"] > batch_data["close"]).astype(int)
    
    
    horizons = [2,5,60,250,1000]
    date_copy=batch_data['date_copy'].copy()
    batch_data = batch_data.drop(columns=['date_copy'])

    for horizon in horizons:
        rolling_averages = batch_data.rolling(horizon).mean()

        ratio_column = f"close_ratio_{horizon}"
        batch_data[ratio_column] = batch_data["close"] / rolling_averages["close"]

        trend_column = f"trend_{horizon}"
        batch_data[trend_column] = batch_data.shift(1).rolling(horizon).sum()["target"]

        #new_predictors+= [ratio_column, trend_column]

    batch_data['date_copy']=date_copy.copy()
    batch_data = batch_data.dropna(subset=batch_data.columns[batch_data.columns != "tomorrow"])
    new_predictors = ['close_ratio_2','trend_2','close_ratio_5','trend_5','close_ratio_60','trend_60','close_ratio_250','trend_250','close_ratio_1000','trend_1000','close','volume','open','high','low']
    
    y_pred = model.predict_proba(batch_data[new_predictors])[:,1]
    y_pred[y_pred >=.6] = 1
    y_pred[y_pred <.6] = 0
    #print(y_pred)
    offset = 1
    y_pred = pd.Series(y_pred, index=batch_data.index, name="predictions")
    target = y_pred[y_pred.size-offset]
    target_url = "https://raw.githubusercontent.com/Deepak0197/id2223_Project_sp500_stocks/main/sp500_dataset/" + str(int(target)) + ".png"
    print("sp500 stock direction predicted: " + str(int(target)))

    
    print("Before opening image")
    img_response = requests.get(target_url, stream=True)
    print("After making request")

    # Check the status code of the response
    print("Status code:", img_response.status_code)

    img = Image.open(img_response.raw)
    print("After opening image")

    
    
    img = Image.open(requests.get(target_url, stream=True).raw)            
    img.save("./latest_sp500.png")
    dataset_api = project.get_dataset_api()    
    dataset_api.upload("./latest_sp500.png", "Resources/images", overwrite=True)
   
    iris_fg = fs.get_feature_group(name="sp500", version=1)
    df = iris_fg.read() 
    #print(df)
    label = df.iloc[-offset]["target"]
    label_url = "https://raw.githubusercontent.com/Deepak0197/id2223_Project_sp500_stocks/main/sp500_dataset/" + str(int(label)) + ".png"
    print("sp500 actual stock direction: " + str(int(label)))
    img = Image.open(requests.get(label_url, stream=True).raw)            
    img.save("./actual_sp500.png")
    dataset_api.upload("./actual_sp500.png", "Resources/images", overwrite=True)
    
    monitor_fg = fs.get_or_create_feature_group(name="sp500_direction_predictions",
                                                version=1,
                                                primary_key=["datetime"],
                                                description="Sp500 Stock Direction predictions Monitoring"
                                                )
    
    now = datetime.now().strftime("%m/%d/%Y, %H:%M:%S")
    data = {
        'prediction': [target],
        'label': [label],
        'datetime': [now],
       }
    monitor_df = pd.DataFrame(data)
    monitor_fg.insert(monitor_df, write_options={"wait_for_job" : False})
    
    history_df = monitor_fg.read()
    # Add our prediction to the history, as the history_df won't have it - 
    # the insertion was done asynchronously, so it will take ~1 min to land on App
    history_df = pd.concat([history_df, monitor_df])


    df_recent = history_df.tail(4)
    dfi.export(df_recent, './df_recent.png', table_conversion = 'matplotlib')
    dataset_api.upload("./df_recent.png", "Resources/images", overwrite=True)
    
    predictions = history_df[['prediction']]
    labels = history_df[['label']]
    
    print("history_df: ", history_df)

    # Only create the confusion matrix when our wine_quality_predictions feature group has examples of all 6 values - 3 to 8.
    print("Number of different sp500 predictions to date: " + str(predictions.value_counts().count()))

if __name__ == "__main__":
    if LOCAL == True :
        g()
    else:
        with stub.run():
            f()


Connection closed.
Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/210070
Connected. Call `.close()` to terminate connection gracefully.
Connected. Call `.close()` to terminate connection gracefully.
Finished: Reading data from Hopsworks, using ArrowFlight (2.13s) 
sp500 stock direction predicted: 0
Before opening image
After making request
Status code: 200
After opening image


Uploading: 100.000%|███████████████| 190667/190667 elapsed<00:02 remaining<00:00


Finished: Reading data from Hopsworks, using ArrowFlight (1.59s) 
sp500 actual stock direction: 1


Uploading: 100.000%|███████████████| 213518/213518 elapsed<00:01 remaining<00:00
Uploading Dataframe: 100.00% |█| Rows 1/1 | Elapsed Time: 00:06 | Remaining Time


Launching job: sp500_direction_predictions_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/210070/jobs/named/sp500_direction_predictions_1_offline_fg_materialization/executions
Finished: Reading data from Hopsworks, using ArrowFlight (1.08s) 


Uploading: 100.000%|█████████████████| 22722/22722 elapsed<00:01 remaining<00:00

history_df:     prediction  label              datetime
0         0.0      1  01/05/2024, 16:27:01
1         0.0      1  01/07/2024, 11:44:41
2         0.0      1  01/07/2024, 11:45:14
3         0.0      1  01/07/2024, 11:46:45
4         0.0      1  01/07/2024, 11:46:07
5         0.0      1  01/07/2024, 11:45:40
6         0.0      1  01/07/2024, 11:52:37
0         0.0      1  01/07/2024, 23:11:37
Number of different sp500 predictions to date: 1
